# Using NeuNetS Generated Model to Perform Inference on Sample Images with the Watson Machine Learning Client

This notebook demonstrates inferencing with a **Keras** (TensorFlow) model which was designed and trained using *NeuNetS*. *NeuNetS* designs and trains a custom neural network, which users may then download. 

In addition to downloading the model for use elsewhere, the user has the option of deploying the model straight to their Watson Machine Learning instance through the *NeuNetS* interface. Once this is done, they will receive a URL which they can use to classify new images. 

The user can input **raw .jpg or .png images**. 

## 1. Launch Watson Machine Learning Client
First, install the latest version of the Watson Machine Learning Client. 

In [ ]:
!pip install --upgrade watson-machine-learning-client pillow numpy==1.15.4

**Tip**: Restart the kernel (**Kernel** -> **Restart**)

Next import the API Client.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Credentials are necessary to access the user WML instance. 

**Tip**: Authentication information (your credentials) can be found in the Service credentials tab of the WML service instance that you created on IBM Cloud. If there are no credentials listed for your instance in Service credentials, click New credential (+) and enter the information required to generate new authentication information.

In [ ]:
wml_credentials = {
  "url": "<>",
  "access_key": "<>",
  "username": "<>",
  "password": "<>",
  "instance_id": "<>"
}

Now initialize the client with your credentials. 

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

## 2. Load Image to be Classified

Load the .jpg or .png image. This requires the Python Imaging Library. 

In [ ]:
from PIL import Image

Now load the image to be classified. 

In [ ]:
test_image = Image.open('/path_to_image/test-image.jpg')
display(test_image)

In order to score with WML, we convert the processed image into a list and insert it in a payload dictionary. We then pass this dictionary to the scoring URL. This will return the class predicted by the *NeuNetS* model. To convert the image format to a list will require the Numpy library. 

In [ ]:
import numpy as np
image_as_array = np.asarray(test_image)
scoring_payload = {'values': [image_as_array.tolist()]}

## 3. Call the WML Client to Classify the Image

The WML deployment for our model includes a URL endpoint to perform the scoring. Copy this URL here so that the client can call the deployed *NeuNetS* model. 

**Tip:** To access the scoring URL, you must first have a deployed model in WML. After synthesizing a model, the "Deploy Model to Watson Machine Learning" button in the top right hand side of the page will be active. Click this and follow the link to the deployment details. On this page, go to the "Implementation tab". You will need to paste the scoring url that is listed here into the cell below. 

In [ ]:
scoring_url = ""
print(scoring_url)

In [ ]:
result = client.deployments.score(scoring_url, scoring_payload)
result

## 4. Manually Uploading Model to WML

If a user downloads and model and wishes to upload it to WML manually, via the WML Python SDK, they may follow the steps below. 

1. Navigate to the model details page in Watson Studio. 
2. Click the Download Model button in the top right. 
3. Import WML credentials and initialize the WML client as described above.
4. Using the following metadata:

    ```ks_model_metadata = {
    client.repository.ModelMetaNames.NAME: "nnet_model",
    client.repository.ModelMetaNames.DESCRIPTION: "nnets",
    client.repository.ModelMetaNames.AUTHOR_NAME: "neunets",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5",
    client.repository.ModelMetaNames.RUNTIME_NAME: "python",
    client.repository.ModelMetaNames.RUNTIME_VERSION: "3.5",
    'frameworkLibraries': [{"name": "keras", "version": "2.1.5"}]
    }```
    
    store the model in WML, with the command:
    
    ```model_details = client.repository.store_model(model=os.path.join(model_location,model_file),     meta_props=ks_model_metadata, training_data=None)```
    
5. Get the unique model id assigned by WML:  `model_uid = client.repository.get_model_uid(model_details)`
6. Deploy the model in WML:  `deployment_details = client.deployments.create(model_uid, 'my neunets model')`
7. Get the model's scoring URL from WML:  `scoring_url = client.deployments.get_scoring_url(deployment_details)`
8. Make the test sentence into a scoring payload:  `scoring_payload = {'values': [np.asarray(test_image).tolist()]}`
9. Obtain a prediction for the scoring payload:  `result = client.deployments.score(scoring_url, scoring_payload)`

### Author: *NeuNetS* Team

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.